##### Copyright 2021 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Copyright 2021 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/hub/tutorials/wav2vec2_saved_model_finetuning"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/wav2vec2_saved_model_finetuning.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/hub/blob/master/examples/colab/wav2vec2_saved_model_finetuning.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub で表示</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/hub/examples/colab/wav2vec2_saved_model_finetuning.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
  <td>     <a href="https://tfhub.dev/vasudevgupta7/wav2vec2/1"><img src="https://www.tensorflow.org/images/hub_logo_32px.png"> TF Hub モデルを参照</a>
</td>
</table>

# LM ヘッドを使用した Wav2Vec2 の微調整

このノートブックでは、事前にトレーニングされた wav2vec2 モデルを [TFHub](https://tfhub.dev) からロードし、事前にトレーニングされたモデルの上に言語モデリングヘッド（LM）を追加することにより、[LibriSpeech データセット](https://huggingface.co/datasets/librispeech_asr)で微調整します。基礎となるタスクは、**自動音声認識**のモデルを構築することです。つまり、音声が与えられた場合、モデルはそれをテキストに変換できる必要があります。

## セットアップ

このノートブックを実行する前に、GPU ランタイムを使用していることを確認してください（`Runtime` &gt; `Change runtime type` &gt; `GPU`）。次のセルは、[`gsoc-wav2vec2`](https://github.com/vasudevgupta7/gsoc-wav2vec2) パッケージとその依存関係をインストールします。

In [ ]:
!pip3 install -q git+https://github.com/vasudevgupta7/gsoc-wav2vec2@main
!sudo apt-get install -y libsndfile1-dev
!pip3 install -q SoundFile

## `TFHub` を使用したモデルのセットアップ

まず、いくつかのライブラリ/モジュールをインポートします。

In [ ]:
import os

import tensorflow as tf
import tensorflow_hub as hub
from wav2vec2 import Wav2Vec2Config

config = Wav2Vec2Config()

print("TF version:", tf.__version__)

最初に TFHub からモデルをダウンロードし、モデルの署名を [`hub.KerasLayer`](https://www.tensorflow.org/hub/api_docs/python/hub/KerasLayer) でラップして、他の Keras レイヤーと同じようにこのモデルを使用できるようにします。幸い、`hub.KerasLayer` はたった1行で両方を実行できます。

**注意: **`hub.KerasLayer` を使用してモデルをロードすると、モデルは少し不透明になりますが、モデルをより細かく制御する必要がある場合は、 `tf.keras.models.load_model(...)` を使用してモデルをロードできます。

In [ ]:
pretrained_layer = hub.KerasLayer("https://tfhub.dev/vasudevgupta7/wav2vec2/1", trainable=True)

モデルエクスポートスクリプトに興味がある場合は、この[スクリプト](https://github.com/vasudevgupta7/gsoc-wav2vec2/blob/main/src/export2hub.py)を参照できます。オブジェクト `pretrained_layer` は、[`Wav2Vec2Model`](https://github.com/vasudevgupta7/gsoc-wav2vec2/blob/main/src/wav2vec2/modeling.py) のフリーズバージョンです。これらの事前トレーニング済みの重みは、[このスクリプト](https://huggingface.co/facebook/wav2vec2-base)を使用して HuggingFacePyTorch[ 事前トレーニング済み](https://github.com/vasudevgupta7/gsoc-wav2vec2/blob/main/src/convert_torch_to_tf.py)の重みから変換されました。

もともと、wav2vec2 はマスクされた時間ステップの真の量子化された潜在的な音声表現を識別することを目的として、マスクされた言語モデリングアプローチで事前にトレーニングされていました。トレーニングの目的について詳しくは、[wav2vec 2.0: A Framework for Self-Supervised Learning of Speech Representations](https://arxiv.org/abs/2006.11477) をお読みください。

ここで、次のセルで役立つ定数とハイパーパラメータをいくつか定義します。モデルのシグネチャは静的シーケンス長 `246000` のみを受け入れるため、`AUDIO_MAXLEN` は意図的に `246000` に設定されています。

In [ ]:
AUDIO_MAXLEN = 246000
LABEL_MAXLEN = 256
BATCH_SIZE = 2

次のセルでは、`pretrained_layer` と高密度レイヤー（LM ヘッド）を [Keras の Functional API](https://www.tensorflow.org/guide/keras/functional) でラップします。

In [ ]:
inputs = tf.keras.Input(shape=(AUDIO_MAXLEN,))
hidden_states = pretrained_layer(inputs)
outputs = tf.keras.layers.Dense(config.vocab_size)(hidden_states)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

各タイムステップで語彙内の各トークンの確率を予測するため、高密度レイヤー（上記で定義）の出力ディメンションは `vocab_size` です。

## トレーニング状態のセットアップ

TensorFlow では、`model.call` または `model.build` が初めて呼び出されたときにのみモデルの重みが作成されるため、次のセルでモデルの重みが作成されます。さらに、`model.summary()` を実行して、トレーニング可能なパラメータの総数を確認します。

In [ ]:
model(tf.random.uniform(shape=(BATCH_SIZE, AUDIO_MAXLEN)))
model.summary()

次に、モデルをトレーニングできるように `loss_fn` とオプティマイザを定義する必要があります。次のセルで定義されます。簡単にするため、`Adam` オプティマイザを使用します。`CTCLoss` は、入力サブパーツを出力サブパーツと簡単に位置合わせできないタスク（`ASR` など）に使用される一般的な損失タイプです。 CTC-loss の詳細については、このすばらしい[ブログ投稿](https://distill.pub/2017/ctc/)をご覧ください。

（[`gsoc-wav2vec2`](https://github.com/vasudevgupta7/gsoc-wav2vec2)パッケージからの）`CTCLoss`は、`config`、`model_input_shape`、`division_factor` の 3 つの引数を受け入れます。`division_factor=1` の場合、損失は単純に合計されるため、それに応じて `division_factor` を渡して、バッチ全体の平均を取得します。

In [ ]:
from wav2vec2 import CTCLoss

LEARNING_RATE = 5e-5

loss_fn = CTCLoss(config, (BATCH_SIZE, AUDIO_MAXLEN), division_factor=BATCH_SIZE)
optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)

## データの読み込みと前処理

LibriSpeech データセットを[公式ウェブサイト](http://www.openslr.org/12)からダウンロードして設定しましょう。

In [ ]:
!wget https://www.openslr.org/resources/12/dev-clean.tar.gz -P ./data/train/
!tar -xf ./data/train/dev-clean.tar.gz -C ./data/train/

**注意: **このノートブックはデモンストレーションのみを目的としているため、`dev-clean` 構成を使用しており、少量のデータが必要です。完全なトレーニングデータは、[LibriSpeech Webサイト](http://www.openslr.org/12)から簡単にダウンロードできます。

In [ ]:
ls ./data/train/

データセットは LibriSpeech ディレクトリにあります。これらのファイルを調べてみましょう。

In [ ]:
data_dir = "./data/train/LibriSpeech/dev-clean/2428/83705/"
all_files = os.listdir(data_dir)

flac_files = [f for f in all_files if f.endswith(".flac")]
txt_files = [f for f in all_files if f.endswith(".txt")]

print("Transcription files:", txt_files, "\nSound files:", flac_files)

なるほど。各サブディレクトリには、多くの `.flac` ファイルと `.txt` ファイルがあります。`.txt` ファイルには、そのサブディレクトリに存在するすべての音声サンプル（つまり、`.flac` ファイル）のテキスト文字起こしが含まれています。

このテキストデータは次のようにロードできます。

In [ ]:
def read_txt_file(f):
  with open(f, "r") as f:
    samples = f.read().split("\n")
    samples = {s.split()[0]: " ".join(s.split()[1:]) for s in samples if len(s.split()) > 2}
  return samples

同様に、`.flac` ファイルから音声サンプルをロードするための関数を定義します。

wav2vec2 は `16K` の頻度で事前トレーニングされているため、`REQUIRED_SAMPLE_RATE` は `16000` に設定されます。頻度によるデータ分布の大きな変化なしに、微調整することをお勧めします。

In [ ]:
import soundfile as sf

REQUIRED_SAMPLE_RATE = 16000

def read_flac_file(file_path):
  with open(file_path, "rb") as f:
      audio, sample_rate = sf.read(f)
  if sample_rate != REQUIRED_SAMPLE_RATE:
      raise ValueError(
          f"sample rate (={sample_rate}) of your files must be {REQUIRED_SAMPLE_RATE}"
      )
  file_id = os.path.split(file_path)[-1][:-len(".flac")]
  return {file_id: audio}

次に、ランダムなサンプルをいくつか選び、視覚化してみましょう。

In [ ]:
from IPython.display import Audio
import random

file_id = random.choice([f[:-len(".flac")] for f in flac_files])
flac_file_path, txt_file_path = os.path.join(data_dir, f"{file_id}.flac"), os.path.join(data_dir, "2428-83705.trans.txt")

print("Text Transcription:", read_txt_file(txt_file_path)[file_id], "\nAudio:")
Audio(filename=flac_file_path)

次に、すべての音声とテキストのサンプルを組み合わせて、その目的のために（次のセルで）関数を定義します。

In [ ]:
def fetch_sound_text_mapping(data_dir):
  all_files = os.listdir(data_dir)

  flac_files = [os.path.join(data_dir, f) for f in all_files if f.endswith(".flac")]
  txt_files = [os.path.join(data_dir, f) for f in all_files if f.endswith(".txt")]

  txt_samples = {}
  for f in txt_files:
    txt_samples.update(read_txt_file(f))

  speech_samples = {}
  for f in flac_files:
    speech_samples.update(read_flac_file(f))

  assert len(txt_samples) == len(speech_samples)

  samples = [(speech_samples[file_id], txt_samples[file_id]) for file_id in speech_samples.keys() if len(speech_samples[file_id]) < AUDIO_MAXLEN]
  return samples

いくつかのサンプルを見てみましょう...

In [ ]:
samples = fetch_sound_text_mapping(data_dir)
samples[:5]

注意: このノートブックで少量のデータセットを操作する際に、このデータはメモリに読み込まれます。ただし、完全なデータセット（〜300 GB）でトレーニングするには、データを遅延ロードする必要があります。[このスクリプト](https://github.com/vasudevgupta7/gsoc-wav2vec2/blob/main/src/data_utils.py)を参照して、詳細を確認できます。

今すぐデータを前処理しましょう!!!

まず、`gsoc-wav2vec2` パッケージを使用してトークナイザーとプロセッサーを定義します。次に、非常に簡単な前処理を行います。`processor` はフレーム軸に対して生の音声を正規化し、`tokenizer` はモデル出力を文字列に変換し（定義された語彙を使用）、特別なトークンを削除します（トークナイザーの構成によって異なります）。

In [ ]:
from wav2vec2 import Wav2Vec2Processor
tokenizer = Wav2Vec2Processor(is_tokenizer=True)
processor = Wav2Vec2Processor(is_tokenizer=False)

def preprocess_text(text):
  label = tokenizer(text)
  return tf.constant(label, dtype=tf.int32)

def preprocess_speech(audio):
  audio = tf.constant(audio, dtype=tf.float32)
  return processor(tf.transpose(audio))

次に、上記のセルで定義した前処理関数を呼び出す Python ジェネレーターを定義します。

In [ ]:
def inputs_generator():
  for speech, text in samples:
    yield preprocess_speech(speech), preprocess_text(text)

## `tf.data.Dataset` のセットアップ

次のセルは、`.from_generator(...)` メソッドを使用して `tf.data.Dataset` オブジェクトをセットアップします。上記のセルで定義した `generator` オブジェクトを使用します。

**注意: **分散トレーニング（特に TPU）の場合、`.from_generator(...)` は現在機能しないため、`.tfrecord` 形式で保存されたデータでトレーニングすることをお勧めします（注意: TFRecord は、TPU が最大限に機能するように、理想的には GCS バケット内に保存する必要があります）。

LibriSpeech データを tfrecords に変換する方法の詳細については、[このスクリプト](https://github.com/vasudevgupta7/gsoc-wav2vec2/blob/main/src/make_tfrecords.py) を参照してください。

In [ ]:
output_signature = (
    tf.TensorSpec(shape=(None),  dtype=tf.float32),
    tf.TensorSpec(shape=(None), dtype=tf.int32),
)

dataset = tf.data.Dataset.from_generator(inputs_generator, output_signature=output_signature)

In [ ]:
BUFFER_SIZE = len(flac_files)
SEED = 42

dataset = dataset.shuffle(BUFFER_SIZE, seed=SEED)

データセットを複数のバッチに渡すため、次のセルでバッチを準備しましょう。ここで、バッチ内のすべてのシーケンスを一定の長さにパディングする必要があります。そのために `.padded_batch(...)` メソッドを使用します。

In [ ]:
dataset = dataset.padded_batch(BATCH_SIZE, padded_shapes=(AUDIO_MAXLEN, LABEL_MAXLEN), padding_values=(0.0, 0))

アクセラレータ（GPU/TPU など）は非常に高速であり、データの読み込み部分が CPU で発生するため、トレーニング中にデータの読み込み（および前処理）がボトルネックになることがよくあります。これにより、特に多くのオンライン前処理が含まれる場合や、データが GCS バケットからオンラインでストリーミングされる場合に、トレーニング時間が大幅に増加する可能性があります。これらの問題に対応するために、`tf.data.Dataset` は `.prefetch(...)` メソッドを提供します。この方法は、モデルが現在のバッチで（GPU/TPU で）予測を行っている間に、次のいくつかのバッチを（CPU で）並行して準備するのに役立ちます。

In [ ]:
dataset = dataset.prefetch(tf.data.AUTOTUNE)

このノートブックはデモンストレーションを目的に作成されているため、最初に `num_train_batches` を取得し、それだけでトレーニングを実行します。ただし、データセット全体でトレーニングすることをお勧めします。同様に、`num_val_batches` のみを評価します。

In [ ]:
num_train_batches = 10
num_val_batches = 4

train_dataset = dataset.take(num_train_batches)
val_dataset = dataset.skip(num_train_batches).take(num_val_batches)

## モデルのトレーニング

モデルをトレーニングするため、モデルを `.compile(...)` でコンパイルした後に、`.fit(...)` メソッドを直接呼び出します。

In [ ]:
model.compile(optimizer, loss=loss_fn)

上記のセルは、トレーニング状態を設定します。これで、`.fit(...)` メソッドを使用してトレーニングを開始できます。

In [ ]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=3)
history.history

後で推論を実行できるように、`.save(...)` メソッドを使用してモデルを保存しましょう。[TFHub のドキュメント](https://www.tensorflow.org/hub/publish)に従って、この SavedModel を TFHub にエクスポートすることもできます。

In [ ]:
save_dir = "finetuned-wav2vec2"
model.save(save_dir, include_optimizer=False)

注意: このモデルは推論のみに使用するため、`include_optimizer=False` を設定しています。

## 評価

次に、検証データセットの単語誤り率を計算します

**単語誤り率**（WER）は、自動音声認識システムのパフォーマンスを測定するための一般的な指標です。WER は、単語レベルで機能するレーベンシュタイン距離から導出されます。単語誤り率は WER = (S + D + I) / N = (S + D + I) / (S + D + C) で計算でき、S は置換の数、D は削除の数、I は挿入の数、C は正しい単語の数、N は参照内の単語の数です（N=S+D+C）。この値は、誤って予測された単語の割合を示します。

WER の詳細については、[この論文](https://www.isca-speech.org/archive_v0/interspeech_2004/i04_2765.html)を参照してください。

[HuggingFace データセット](https://huggingface.co/docs/datasets/)ライブラリの `load_metric(...)` 関数を使用します。最初に `pip` を使用して `datasets` ライブラリをインストールしてから、`metric`オブジェクトを定義しましょう。

In [ ]:
!pip3 install -q datasets

from datasets import load_metric
metric = load_metric("wer")

In [ ]:
@tf.function(jit_compile=True)
def eval_fwd(batch):
  logits = model(batch, training=False)
  return tf.argmax(logits, axis=-1)

次に、検証データの評価を実行します。

In [ ]:
from tqdm.auto import tqdm

for speech, labels in tqdm(val_dataset, total=num_val_batches):
    predictions  = eval_fwd(speech)
    predictions = [tokenizer.decode(pred) for pred in predictions.numpy().tolist()]
    references = [tokenizer.decode(label, group_tokens=False) for label in labels.numpy().tolist()]
    metric.add_batch(references=references, predictions=predictions)

`tokenizer.decode(...)` メソッドを使用して、予測とラベルをテキストにデコードし直し、後で `WER` 計算用のメトリックに追加します。

では、次のセルでメトリック値を計算しましょう。

In [ ]:
metric.compute()

**注意: **モデルは非常に小さなデータでトレーニングされており、ASR のようなタスクでは、音声からテキストへのマッピングを学習するために大量のデータが必要になることが多いため、ここでのメトリック値は意味がありません。良い結果を得るには、おそらく大きなデータでトレーニングする必要があります。このノートブックは、事前にトレーニングされた音声モデルを微調整するためのテンプレートを提供します。

## 推論

トレーニングプロセスに満足し、モデルを `save_dir` に保存したので、このモデルを推論に使用する方法を確認します。

まず、`tf.keras.models.load_model(...)` を使用してモデルをロードします。

In [ ]:
finetuned_model = tf.keras.models.load_model(save_dir)

推論を実行するための音声サンプルをダウンロードしましょう。次のサンプルを音声サンプルに置き換えることもできます。

In [ ]:
!wget https://github.com/vasudevgupta7/gsoc-wav2vec2/raw/main/data/SA2.wav

`soundfile.read(...)` を使用して音声サンプルを読み取り、モデルのシグネチャを満たすために `AUDIO_MAXLEN` にパディングします。次に、<br>`Wav2Vec2Processor` インスタンスを使用してその音声サンプルを正規化し、モデルにフィードします。

In [ ]:
import numpy as np

speech, _ = sf.read("SA2.wav")
speech = np.pad(speech, (0, AUDIO_MAXLEN - len(speech)))
speech = tf.expand_dims(processor(tf.constant(speech)), 0)

outputs = finetuned_model(speech)
outputs

上で定義した `Wav2Vec2tokenizer` インスタンスを使用して、数値をデコードしてテキストシーケンスに戻しましょう。

In [ ]:
predictions = tf.argmax(outputs, axis=-1)
predictions = [tokenizer.decode(pred) for pred in predictions.numpy().tolist()]
predictions

モデルはこのノートブックの大きなデータでトレーニングされたことがないため、この予測は非常にランダムです（このノートブックは完全なトレーニングを行うためのものではないためです）。このモデルを完全な LibriSpeech データセットでトレーニングすると、適切な予測が得られます。

ようやくこのノートブックの最後にたどり着きました。しかし、音声関連のタスクについての TensorFlow の学習はこれで終わりではありません。この[リポジトリ](https://github.com/tulasiram58827/TTS_TFLite)には、さらにすばらしいチュートリアルがいくつか含まれています。このノートブックでバグが発生した場合は、[ここ](https://github.com/vasudevgupta7/gsoc-wav2vec2/issues)で問題を作成してください。